# 06_02_engineering_reusable_pipelines.ipynb

# Engineering Reusable Pipelines
- Create reusable preprocessing classes
- Compare Python-level vs model-level preprocessing
- Understand deployment implications
- Save and reload a model safely

In [1]:
import tensorflow as tf
import tensorflow_datasets as tfds
import numpy as np

print("TensorFlow version:", tf.__version__)

# Load dataset
(ds_train, ds_val), ds_info = tfds.load(
    "tf_flowers",
    split=["train[:80%]", "train[80%:]"],
    as_supervised=True,
    with_info=True
)

num_classes = ds_info.features["label"].num_classes
class_names = ds_info.features["label"].names


TensorFlow version: 2.9.1


In [2]:
# Create a Reusable Preprocessor Class 

class ImagePreprocessor:
    def __init__(self, img_size=(160, 160)):
        self.img_size = img_size

    def resize(self, image):
        return tf.image.resize(image, self.img_size)

    def normalize(self, image):
        return image / 255.0

    def __call__(self, image, label):
        image = self.resize(image)
        image = self.normalize(image)
        return image, label


In [3]:
# Apply it

preprocessor = ImagePreprocessor()

train_ds = (
    ds_train
    .map(preprocessor)
    .shuffle(1000)
    .batch(32)
    .prefetch(tf.data.AUTOTUNE)
)


## Discussion
- What happens if someone forgets to use this class at inference time?
- Is this guaranteed to be applied in production?

In [4]:
# Convert Preprocessing to Keras Layers

preprocessing_block = tf.keras.Sequential([
    tf.keras.layers.Resizing(160, 160),
    tf.keras.layers.Rescaling(1./255)
])


In [5]:
IMG_SIZE = (160, 160)

# Preprocessing block inside the model
preprocessing_block = tf.keras.Sequential([
    tf.keras.layers.Resizing(160, 160),
    tf.keras.layers.Rescaling(1./255)
])

# Base model (CPU friendly)
base_model = tf.keras.applications.MobileNetV2(
    input_shape=(160, 160, 3),
    include_top=False,
    weights="imagenet"
)

base_model.trainable = False

# Full model
model_with_preprocessing = tf.keras.Sequential([
    preprocessing_block,
    base_model,
    tf.keras.layers.GlobalAveragePooling2D(),
    tf.keras.layers.Dense(num_classes, activation="softmax")
])

model_with_preprocessing.compile(
    optimizer="adam",
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"]
)


In [6]:
# Train for 1 epoch before saving

train_raw = (
    ds_train
    .map(lambda x, y: (tf.image.resize(x, IMG_SIZE), y))
    .shuffle(1000)
    .batch(32)
    .prefetch(tf.data.AUTOTUNE)
)

val_raw = (
    ds_val
    .map(lambda x, y: (tf.image.resize(x, IMG_SIZE), y))
    .batch(32)
    .prefetch(tf.data.AUTOTUNE)
)

model_with_preprocessing.fit(
    train_raw,
    validation_data=val_raw,
    epochs=1
)


92/92 [==============================] - 17s 145ms/step - loss: 0.7740 - accuracy: 0.7095 - val_loss: 0.4680 - val_accuracy: 0.8379


In [7]:
# Save and Reload Model

model_with_preprocessing.save("flower_model")

reloaded_model = tf.keras.models.load_model("flower_model")


INFO:tensorflow:Assets written to: flower_model\assets


INFO:tensorflow:Assets written to: flower_model\assets


In [10]:
# Test inference on a raw image:

reloaded_model = tf.keras.models.load_model("flower_model")

for image, label in ds_val.take(1):
    resized = tf.image.resize(image, (160, 160))
    prediction = reloaded_model(tf.expand_dims(resized, 0))
    print("Predicted class:", class_names[np.argmax(prediction)])


Predicted class: tulips


# Discussion
- Embedding preprocessing in the model ensures:
  - Reproducibility
  - Portability
  - Production safety